In [ ]:
# 以下のtrainデータから特長量を作成し、学習モデルを生成させる。
# /mnt/c/Users/hayat/Desktop/keiba_analysis/data_for_train/train/2005_2022

: 

In [ ]:
!pip install lightGBM

In [ ]:
import gc
gc.collect()

In [ ]:
# /mnt/c/Users/hayat/Desktop/keiba_analysis/data_for_train/train/2005_2022
# のデータを取り込む

import warnings
from sklearn.svm import SVC, LinearSVC
from sklearn import multiclass
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.utils.class_weight import compute_sample_weight
import scipy.stats
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import tqdm

In [ ]:
inference_page_id = 2025050108

is_denso = False
# train_year_list = ["2024"]
train_year_list = ["2023", "2024"]
train_data = pd.DataFrame()
horse_past_data = pd.DataFrame()
horse_peds_data = pd.DataFrame()
jockey_past_data = pd.DataFrame()
for year in train_year_list:
    train_path = '/home/hayato/horse_inference/data_for_train/train_data/' + year + '/'
    if is_denso:
        train_path = '/home/denso/horse_inference/data_for_train/train_data/' + year + '/'
    # train dataに各年の学習データを縦方向に結合
    train_data = pd.concat([train_data, pd.read_csv(train_path + "train_data_results_" + year + ".csv", encoding='utf-8')], axis=0)
    print("train data length {}".format(len(train_data)))

    # 勝率データ、過去のデータを取得
    horse_past_data = pd.concat([horse_past_data, pd.read_csv(train_path + 'horse_results_' + year + '.csv', encoding='utf-8')], axis=0)
    print("horse_past_data length {}".format(len(horse_past_data)))
    horse_peds_data = pd.concat([horse_peds_data, pd.read_csv(train_path + 'horse_born_results_' + year + '.csv', encoding='utf-8')], axis=0)
    print("horse_peds_data length {}".format(len(horse_peds_data)))
    # ジョッキーの過去データを取得
    jockey_past_data = pd.concat([jockey_past_data, pd.read_csv(train_path + 'jockey_results_' + year + '.csv', encoding='utf-8')], axis=0)
    # jockeyデータはデータの中で"累計"が含まれているものだけを抽出
    # "年度"列が"累計"の行だけを抽出
    jockey_past_data = jockey_past_data[jockey_past_data['年度'] == '累計']
    print("jockey_past_data length {}".format(len(jockey_past_data)))

In [ ]:
# horse_past_dataから最初のhorse_idの情報だけを抽出
first_horse_past_data = horse_past_data.drop_duplicates(subset='horse_id', keep='first')
# train_data = pd.merge(train_data, horse_peds_data, on='horse_id', how='left')

# train dataに対してhorse_past_data、horse_peds_dataをhorse_idをキーにして結合
train_data = pd.merge(train_data, first_horse_past_data, on='horse_id', how='left')
# train dataの情報を表示
print(train_data.info())
# train dataのデータ長を表示
print("train_data lenght :{}".format(len(train_data)))

In [ ]:
# horse_peds_dataはpeds62がhorse_idなので、peds62をhorse_idに変更
horse_peds_data = horse_peds_data.rename(columns={'peds_62': 'horse_id'})
# すごい馬の血糖情報があるけどそんなにいらないので、5頭分のpeds0からpeds5までの情報だけを取得
horse_peds_data = horse_peds_data[['horse_id', 'peds_0', 'peds_1', 'peds_2', 'peds_3', 'peds_4', 'peds_5']]
# 1pedsに色んな情報が入っているので、半角スペースまでの情報だけを取得
horse_peds_data['peds_0'] = horse_peds_data['peds_0'].apply(lambda x: x.split(' ')[0])
horse_peds_data['peds_1'] = horse_peds_data['peds_1'].apply(lambda x: x.split(' ')[0])
horse_peds_data['peds_2'] = horse_peds_data['peds_2'].apply(lambda x: x.split(' ')[0])
horse_peds_data['peds_3'] = horse_peds_data['peds_3'].apply(lambda x: x.split(' ')[0])
horse_peds_data['peds_4'] = horse_peds_data['peds_4'].apply(lambda x: x.split(' ')[0])
horse_peds_data['peds_5'] = horse_peds_data['peds_5'].apply(lambda x: x.split(' ')[0])
# train dataに対してhorse_peds_dataをhorse_idをキーにして結合
train_data = pd.merge(train_data, horse_peds_data, on='horse_id', how='left')
# train dataの情報を表示
print(train_data.info())
# train dataのデータ長を表示
print("train_data lenght :{}".format(len(train_data)))

In [ ]:
# train_dataに対してjockey_past_dataをjockey_idをキーにして結合
# jockey_resultsから必要なjockey_idの情報だけを抽出
jockey_info = jockey_past_data[jockey_past_data['jockey_id'].isin(train_data['jockey_id'])]

train_data = pd.merge(train_data, jockey_info, on='jockey_id', how='left')
# train_data = pd.merge(train_data, jockey_past_data, on='jockey_id', how='left')
print(train_data.info())
# テスト的にtrain_dataのデータをcsvに保存
train_data.to_csv(train_path + "train_data_check.csv", index=False, encoding='utf-8')
print("train_data lenght :{}".format(len(train_data)))
train_data.describe()



In [ ]:
# train_dataの重複をrace_id, horse_idで削除
train_data = train_data.drop_duplicates(subset=["race_id", "horse_id"], keep='first')
# 重複をassertで確認
assert train_data.duplicated(subset=["race_id", "horse_id"]).sum() == 0

# oddsだけ前処理の前に準備しておく
train_data['単勝'] = train_data['単勝'].replace('---', 100)
train_data['odds'] = train_data['単勝'].apply(lambda x: float(x))   
train_data_for_ranking = train_data.copy()


In [ ]:

inference_data_path = "/home/hayato/horse_inference/inference/" + str(inference_page_id) + "/add_dynamic_data/"
if is_denso:
    inference_data_path = "/home/denso/horse_inference/inference/" + str(inference_page_id) + "/add_dynamic_data/"

In [ ]:
train_data.info()
train_data_mlp = train_data.copy()


In [ ]:
from utils.utils import *

train_data = preprocess(train_data)
# mlp用にコピーする
train_data_mlp = preprocess(train_data_mlp, is_mlp=True)   


In [ ]:
# レースごとのオッズの特長量変換
# 追加する特長量
# log_odds: オッズの対数
# normalized_odds: オッズの正規化
# zscore_odds: オッズのzscore
# odds_rank: オッズの順位
# odds_std: オッズの標準偏差
train_data_for_ranking = odds_feature_engineering(train_data_for_ranking)


In [ ]:
# 各レースごとにオッズをソートして1位、2位、3位のオッズを取得
# オッズの差分を計算して新しい特徴量として追加
odds_diff_1_2, odds_diff_1_3 = get_odds_differences(train_data_for_ranking)
train_data_for_ranking['odds_diff-1-2'] = odds_diff_1_2
train_data_for_ranking['odds_diff-1-3'] = odds_diff_1_3

# 結果を確認
# print(train_data_for_ranking[['race_id', 'odds', 'odds_diff_1_2', 'odds_diff_1_3']].head())

In [ ]:
# ランキング学習での推論
# step1 クエリを作成する
# ランキング学習のためのクエリを作成

# ============データ情報整理================
# X_train: 学習用データ。クエリ必要
# X_valid: 検証データ。クエリ必要
# y_train: 学習用データの答え
# y_valid: 検証データの答え
# test_total: テストデータ。クエリ不要
# y_test_true: テストデータの答え
# ========================================
# ランキング学習用のtrain dataを7:3で分割。この際データがばらばらに混ざってしまうので上から7割を学習データ、残りをテストデータとする
train_ranking = train_data_for_ranking[:int(len(train_data_for_ranking)*0.7)]
test_ranking = train_data_for_ranking[int(len(train_data_for_ranking)*0.7):]
test_ranking_race_id = test_ranking["race_id"]
# train_rankingを7:3で分割。この際データがばらばらに混ざってしまうので上から7割を学習データ、残りをテストデータとする
train_ranking = train_ranking[:int(len(train_ranking)*0.7)]
valid_ranking = train_ranking[int(len(train_ranking)*0.7):]

# trainデータのクエリを作成
train_query, sorted_train_data = create_query(train_ranking,"train")
# validデータのクエリを作成
valid_query, sorted_valid_data = create_query(valid_ranking,"valid")




In [ ]:
# クエリの数の合計を確認
print("Number of queries in train data: {}".format(sum(train_query)))
print("Number of queries in valid data: {}".format(sum(valid_query)))
print("Total number of queries: {}".format(len(train_query) + len(valid_query)))
print("Number of rows in train data: {}".format(len(sorted_train_data)))
print("Number of rows in valid data: {}".format(len(sorted_valid_data)))
print("Total number of rows: {}".format(len(sorted_train_data) + len(sorted_valid_data)))


In [ ]:
# クエリを取得したので前処理をかける
X_sorted_train_data = preprocess(sorted_train_data,is_ranking=True, is_mlp=True)
X_sorted_valid_data = preprocess(sorted_valid_data,is_ranking=True, is_mlp=True)
test_ranking = preprocess(test_ranking,is_ranking=True, is_mlp=True)

In [ ]:
# スクレイピングするごとにカラム名が変化してくので、カラム名のスペースを削除する処理を学習前に追加
# X_sorted_train_data.columnsの日本語文字列の半角、全角スペースを削除する
X_sorted_train_data.columns = X_sorted_train_data.columns.str.replace(' ', '')
X_sorted_train_data.columns = X_sorted_train_data.columns.str.replace('　', '')

# X_sorted_valid_data.columnsの日本語文字列の半角、全角スペースを削除する
X_sorted_valid_data.columns = X_sorted_valid_data.columns.str.replace(' ', '')
X_sorted_valid_data.columns = X_sorted_valid_data.columns.str.replace('　', '')

# test_ranking.columnsの日本語文字列の半角、全角スペースを削除する
test_ranking.columns = test_ranking.columns.str.replace(' ', '')
test_ranking.columns = test_ranking.columns.str.replace('　', '')

In [ ]:
# 2位以下の予測のため学習データをコピーしておく
X_sorted_train_data_copy = X_sorted_train_data.copy()
X_sorted_valid_data_copy = X_sorted_valid_data.copy()
test_ranking_copy = test_ranking.copy()



In [ ]:
# うまく目的変数の変換ができないので、現状のデータを見てみる
# X_sorted_train_data.to_csv("X_sorted_train_data.csv", index=False)

In [ ]:
# X_sorted_train_dataにgoal_number_replaceを追加してcsvに保存する
# X_sorted_train_data.to_csv("X_sorted_train_data_add_goal_number.csv", index=False)

In [ ]:
# 1. 目的変数goal_number_replaceを確保
# 2. 実際のgoal_numberを確保
# 3. goal_number_replaceを削除
# 4. goal_numberを削除
# 1========================================================
y_train_ranking = X_sorted_train_data['goal_number_replace']
y_valid_ranking = X_sorted_valid_data['goal_number_replace']
y_test_true_ranking = test_ranking['goal_number_replace']
# 2========================================================
y_train_ranking_goal = X_sorted_train_data['goal_number']
y_valid_ranking_goal = X_sorted_valid_data['goal_number']
y_test_true_ranking_goal = test_ranking['goal_number']
# 3========================================================
X_sorted_train_data = X_sorted_train_data.drop('goal_number', axis=1)
X_sorted_valid_data = X_sorted_valid_data.drop('goal_number', axis=1)
test_ranking = test_ranking.drop('goal_number', axis=1)
# 4========================================================
X_sorted_train_data = X_sorted_train_data.drop('goal_number_replace', axis=1)
X_sorted_valid_data = X_sorted_valid_data.drop('goal_number_replace', axis=1)
test_ranking = test_ranking.drop('goal_number_replace', axis=1)
# 5========================================================
X_sorted_train_data = X_sorted_train_data.drop('race_id', axis=1)
X_sorted_valid_data = X_sorted_valid_data.drop('race_id', axis=1)
test_ranking = test_ranking.drop('race_id', axis=1)

In [ ]:
# 学習データ
X_sorted_train_data.info()

In [ ]:
test_ranking.info()

In [ ]:
# クエリの合計とデータの合計が一致しているか確認
print("Number of queries in train data: {}".format(sum(train_query)))
print("Number of queries in valid data: {}".format(sum(valid_query)))
# y_train_rankingの数とX_sorted_train_dataの数が一致しているか確認
print("y_train_ranking length {}".format(len(y_train_ranking)))
print("Number of rows in train data: {}".format(len(X_sorted_train_data)))
print("Number of rows in valid data: {}".format(len(X_sorted_valid_data)))
print("Total number of rows: {}".format(len(X_sorted_train_data) + len(X_sorted_valid_data)))

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
from sklearn.metrics import ndcg_score

# データの準備
# X_sorted_train_data, y_train_ranking, train_query
# X_sorted_valid_data, y_valid_ranking, valid_query

# ランキング学習
model = lgb.LGBMRanker(
    random_state=42,
    objective='lambdarank',
    metric='ndcg',
    n_estimators=500,              # 決定木の個数(default:100) 5000のほうが10000より高い
    learning_rate=0.1,            # 学習率(default:0.1)
    num_leaves=10,                 # 決定木にある分岐の個数(default:31)
    max_depth=10,                  # 決定木の深さの最大値(default:-1)
    min_child_samples=40,         # 決定木のノードに含まれる最小サンプル数(default:20)
    # feature_fraction=0.8,
    # bagging_fraction=0.8,
    # bagging_freq=10,
    # lambda_l1=0.1,
    # lambda_l2=0.1,
    # min_split_gain=0.1,
    # min_child_weight=0.1
    # rambdalank_truncation_level=10,
    max_position=3, # 3位までの順位を予測する,
    # lambdarank_truncation_level = 5,
)

# モデルの学習
model.fit(
    X_sorted_train_data,
    y_train_ranking,
    group=train_query,
    eval_set=[(X_sorted_valid_data, y_valid_ranking)],
    eval_group=[valid_query],
    eval_at=[1, 2, 3],
    eval_metric='ndcg',
)

In [ ]:
# トレーニング結果の取得
evals_result = model.evals_result_

# NDCGスコアのプロット
# NDCGスコアは1になるほど予測モデルの性能が良いことを占める。1,3,5は上位何件の結果を反映するか。
plt.figure(figsize=(10, 6))
plt.plot(evals_result['valid_0']['ndcg@1'], label='NDCG@1')
plt.plot(evals_result['valid_0']['ndcg@2'], label='NDCG@2')
plt.plot(evals_result['valid_0']['ndcg@3'], label='NDCG@3')
plt.xlabel('Iterations')
plt.ylabel('NDCG Score')
plt.title('NDCG Score over Iterations')
plt.legend()
plt.grid(True)
plt.show()
# model.best_iteration_のNDCGを数値で出力
print("Best NDCG@1: {:.4f} in iteration: {}".format(evals_result['valid_0']['ndcg@1'][model.best_iteration_ - 1], model.best_iteration_))
print("Best NDCG@2: {:.4f} in iteration: {}".format(evals_result['valid_0']['ndcg@2'][model.best_iteration_ - 1], model.best_iteration_))
print("Best NDCG@3: {:.4f} in iteration: {}".format(evals_result['valid_0']['ndcg@3'][model.best_iteration_ - 1], model.best_iteration_))

In [ ]:
# アンサンブル候補①LightGBM(RankNet)
# 学習データ： X_sorted_train_data, y_train_ranking
# 学習データのクエリ： train_query
# 検証データ： X_sorted_valid_data, y_valid_ranking
# 検証データのクエリ： valid_query
# テストデータ： test_ranking, y_test_true_ranking

# LightGBMデータセット
train_data = lgb.Dataset(X_sorted_train_data, label=y_train_ranking, group=train_query)
valid_data = lgb.Dataset(X_sorted_valid_data, label=y_valid_ranking, group=valid_query)
# パラメータの設定
params = {
    'objective': 'rank_xendcg',
    'metric': 'ndcg',
    'ndcg_at': [1, 2, 3],
    'n_estimators':1000,              # 決定木の個数(default:100) 5000のほうが10000より高い
    'learning_rate':0.1,            # 学習率(default:0.1)
    'num_leaves':10,                 # 決定木にある分岐の個数(default:31)
    'max_depth':10,                  # 決定木の深さの最大値(default:-1)
    "max_position":3, # 3位までの順位を予測する
    "min_data_in_leaf":40,         # 決定木のノードに含まれる最小サンプル数(default:20)
    # "lambdarank_truncation_level":5,
}

evaluation_results = {}
# モデルの学習
ranknet_model = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    callbacks=[lgb.record_evaluation(evaluation_results)],
)


In [ ]:

# 学習結果の可視化をndcgで行う
# NDCGスコアのプロット
# NDCGスコアは1になるほど予測モデルの性能が良いことを占める。1,3,5は上位何件の結果を反映するか。
plt.figure(figsize=(10, 6))
plt.plot(evaluation_results['valid_0']['ndcg@1'], label='NDCG@1')
plt.plot(evaluation_results['valid_0']['ndcg@2'], label='NDCG@2')
plt.plot(evaluation_results['valid_0']['ndcg@3'], label='NDCG@3')
plt.xlabel('Iterations')
plt.ylabel('NDCG Score')
plt.title('NDCG Score over Iterations')
plt.legend()
plt.grid(True)
plt.show()
# model.best_iteration_のNDCGを数値で出力
print("Best NDCG@1: {:.4f} in iteration: {}".format(ranknet_model.best_score['valid_0']['ndcg@1'], ranknet_model.best_iteration))
# 2も出力
print("Best NDCG@2: {:.4f} in iteration: {}".format(ranknet_model.best_score['valid_0']['ndcg@2'], ranknet_model.best_iteration))
# 3も出力
print("Best NDCG@3: {:.4f} in iteration: {}".format(ranknet_model.best_score['valid_0']['ndcg@3'], ranknet_model.best_iteration))


In [ ]:
# アンサンブル候補②XGBoost(Pairwise)
from xgboost import XGBRanker
# モデルの定義
pairwise_model = XGBRanker(
    objective='rank:pairwise',
    eval_metric='ndcg',
    n_estimators=1000,              # 決定木の個数(default:100) 5000のほうが10000より高い
    learning_rate=0.1,            # 学習率(default:0.1)
    max_depth=10,                    # 決定木の深さの最大値(default:6)
    # subsample=0.8,                  # データのサンプリング比率(default:1)
    # colsample_bytree=0.8,           # 列のサンプリング比率(default:1)
    # min_child_weight=0.5,           # 葉の重みの最小値(default:1)
    # reg_lambda=1.0,                 # L2正則化の強さ(default:1)
    # gamma=0.1,                      # 葉の追加分岐を行うかの閾値(default:0)
    # n_jobs=-1,                      # 並列処理の数(default:1)
    random_state=42,                # 乱数のシード値(default:0)
    max_position=3, # 3位までの順位を予測する
    min_child_samples=30,         # 決定木のノードに含まれる最小サンプル数(default:20)
    # lambdarank_truncation_level=5,
)
# モデルの学習
pairwise_model.fit(
    X_sorted_train_data,
    y_train_ranking,
    group=train_query,
    eval_set=[(X_sorted_valid_data, y_valid_ranking)],
    eval_group=[valid_query],
    verbose=100,
    early_stopping_rounds=100,
)

# 学習結果の可視化をndcgで行う
# NDCGスコアのプロット
# NDCGスコアは1になるほど予測モデルの性能が良いことを占める。1,3,5は上位何件の結果を反映するか。
plt.figure(figsize=(10, 6))
plt.plot(pairwise_model.evals_result()['validation_0']['ndcg'], label='NDCG')
plt.xlabel('Iterations')
plt.ylabel('NDCG Score')
plt.title('NDCG Score over Iterations')
plt.legend()
plt.grid(True)
plt.show()
# model.best_iteration_のNDCGを数値で出力
print("Best NDCG: {:.4f} in iteration: {}".format(pairwise_model.best_score, pairwise_model.best_iteration))

In [ ]:
#　テストデータで推論
# LighGBM(lambdarank)の推論
prediction_test_ranking = model.predict(test_ranking, num_iteration=model.best_iteration_)
# LightGBM(RankNet)の推論
prediction_test_ranking_ranknet = ranknet_model.predict(test_ranking)
# XGBoost(Pairwise)の推論
prediction_test_ranking_pairwise = pairwise_model.predict(test_ranking)
# 上記の平均をアンサンブルモデルとする
prediction_test_ranking_ensemble = (prediction_test_ranking + prediction_test_ranking_ranknet + prediction_test_ranking_pairwise) / 3

df_prediction_test_ranking = pd.DataFrame({
    "馬番号": test_ranking["umaban"],
    "予測スコア(lambdarank)": prediction_test_ranking,
    "予測スコア(RankNet)": prediction_test_ranking_ranknet,
    "予測スコア(Pairwise)": prediction_test_ranking_pairwise,
    "ensemble_prediction": prediction_test_ranking_ensemble,
    "着順関連度": y_test_true_ranking,
    "着順": y_test_true_ranking_goal,
    "race_id": test_ranking_race_id,
    "distance": test_ranking["distance"],
    "ground_state": test_ranking["ground_state"],
    "weather": test_ranking["weather"],
    "condition": test_ranking["condition"],
})


# 同じレースidのうちスコアの大きい順に1位から予測順位を計算する
df_prediction_test_ranking["予測順位(lambdarank)"] = df_prediction_test_ranking.groupby("race_id")["予測スコア(lambdarank)"].rank(ascending=False, method='first')
df_prediction_test_ranking["予測順位(RankNet)"] = df_prediction_test_ranking.groupby("race_id")["予測スコア(RankNet)"].rank(ascending=False, method='first')
df_prediction_test_ranking["予測順位(Pairwise)"] = df_prediction_test_ranking.groupby("race_id")["予測スコア(Pairwise)"].rank(ascending=False, method='first')
df_prediction_test_ranking["ensemble_prediction_rank"] = df_prediction_test_ranking.groupby("race_id")["ensemble_prediction"].rank(ascending=False, method='first')
# それぞれの予測順位はfloat型なのでint型に変換
df_prediction_test_ranking["予測順位(lambdarank)"] = df_prediction_test_ranking["予測順位(lambdarank)"].astype(int)
df_prediction_test_ranking["予測順位(RankNet)"] = df_prediction_test_ranking["予測順位(RankNet)"].astype(int)
df_prediction_test_ranking["予測順位(Pairwise)"] = df_prediction_test_ranking["予測順位(Pairwise)"].astype(int)
df_prediction_test_ranking["ensemble_prediction_rank"] = df_prediction_test_ranking["ensemble_prediction_rank"].astype(int)

# 3つのモデルがすべて1位と判断したときだけ1位とするアンサンブルモデル
# df_prediction_test_ranking['ensemble_prediction'] = (
#     (df_prediction_test_ranking['予測順位(lambdarank)'] == 1) &
#     (df_prediction_test_ranking['予測順位(RankNet)'] == 1) &
#     (df_prediction_test_ranking['予測順位(Pairwise)'] == 1)
# ).astype(int)

# 予測スコアの分布を可視化
plt.figure(figsize=(10, 6))
plt.hist(prediction_test_ranking, bins=50, alpha=0.5, label='LightGBM(lambdarank)')
plt.hist(prediction_test_ranking_ranknet, bins=50, alpha=0.5, label='LightGBM(RankNet)')
plt.hist(prediction_test_ranking_pairwise, bins=50, alpha=0.5, label='XGBoost(Pairwise)')
plt.hist(prediction_test_ranking_ensemble, bins=50, alpha=0.5, label='Ensemble')
plt.xlabel('Predicted Score')
plt.ylabel('Frequency')
plt.title('Predicted Score Distribution')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
# 1位のスコアにおける予測結果の評価
print("LightGBM(lambdarank)の評価")
rank_evaluation(df_prediction_test_ranking, '予測順位(lambdarank)')
print("LightGBM(RankNet)の評価")
rank_evaluation(df_prediction_test_ranking, '予測順位(RankNet)')
print("XGBoost(Pairwise)の評価")
rank_evaluation(df_prediction_test_ranking, '予測順位(Pairwise)')
print("アンサンブルモデルの評価")
rank_evaluation(df_prediction_test_ranking, 'ensemble_prediction_rank')

In [ ]:
# 1位と2位のスコアの差を可視化する→閾値を決定してコンフュージョンマトリックスで正解率を確認する
df_prediction_test_ranking['score_diff_lambdarank'] = calculate_score_diff(df_prediction_test_ranking, 'lambdarank')
df_prediction_test_ranking['score_diff_RankNet'] = calculate_score_diff(df_prediction_test_ranking, 'RankNet')
df_prediction_test_ranking['score_diff_Pairwise'] = calculate_score_diff(df_prediction_test_ranking, 'Pairwise')

plot_score_diff(df_prediction_test_ranking, 'lambdarank', 1)
plot_score_diff(df_prediction_test_ranking, 'RankNet', 1)
plot_score_diff(df_prediction_test_ranking, 'Pairwise', 1)

optimal_threshold_lambdarank, optimal_threshold_pr_lambdarank = find_optimal_threshold(df_prediction_test_ranking, 'lambdarank',1)
optimal_threshold_ranknet, optimal_threshold_pr_ranknet = find_optimal_threshold(df_prediction_test_ranking, 'RankNet',1)
optimal_threshold_pairwise, optimal_threshold_pr_pairwise = find_optimal_threshold(df_prediction_test_ranking, 'Pairwise',1)
print(f'Optimal Threshold (ROC) for lambdarank: {optimal_threshold_lambdarank}')
print(f'Optimal Threshold (Precision-Recall) for lambdarank: {optimal_threshold_pr_lambdarank}')
print(f'Optimal Threshold (ROC) for RankNet: {optimal_threshold_ranknet}')
print(f'Optimal Threshold (Precision-Recall) for RankNet: {optimal_threshold_pr_ranknet}')
print(f'Optimal Threshold (ROC) for Pairwise: {optimal_threshold_pairwise}')
print(f'Optimal Threshold (Precision-Recall) for Pairwise: {optimal_threshold_pr_pairwise}')

df_prediction_test_ranking = apply_threshold(df_prediction_test_ranking, 'lambdarank', optimal_threshold_lambdarank)
df_prediction_test_ranking = apply_threshold(df_prediction_test_ranking, 'RankNet', optimal_threshold_ranknet)
df_prediction_test_ranking = apply_threshold(df_prediction_test_ranking, 'Pairwise', optimal_threshold_pairwise)

evaluate_predictions(df_prediction_test_ranking, 'lambdarank')
evaluate_predictions(df_prediction_test_ranking, 'RankNet')
evaluate_predictions(df_prediction_test_ranking, 'Pairwise')

In [ ]:
# # 1位-2位のスコア/標準偏差を計算でやってみたけどrecallがあがってprecisionが下がる結果に。
# ->未メンテ。
# evaluate_predictions_std(df_prediction_test_ranking, 'lambdarank')
# evaluate_predictions_std(df_prediction_test_ranking, 'RankNet')
# evaluate_predictions_std(df_prediction_test_ranking, 'Pairwise')

In [ ]:
# 予測結果に対して距離、天気、馬場状態、コースの特長量ごとに予測精度が変わらないかを可視化する。
model_evaluation_by_feature(df_prediction_test_ranking, 'lambdarank')
model_evaluation_by_feature(df_prediction_test_ranking, 'RankNet')
model_evaluation_by_feature(df_prediction_test_ranking, 'Pairwise')

# condition
# ['ダ' '芝' '障']
# weather
# ['小雨' '晴' '曇' '雨']
# ground_state
# ['不良' '稍重' '良' '重']

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

   

# 各モデルに対して馬連のprecisionを計算
models = ['予測順位(lambdarank)', '予測順位(Pairwise)', '予測順位(RankNet)']
for model_ in models:
    umaren_precision(df_prediction_test_ranking, model_)

In [ ]:
# 特徴重要度の可視化
df_importance = pd.DataFrame({"columns": X_sorted_train_data.columns, "importance": model.feature_importances_})
df_importance.sort_values("importance", ascending=False, inplace=True)
print(df_importance)

In [ ]:
# カレントディレクトリにモデルを保存
import pickle
with open('pkl_registory/lambdarank_model.pkl', 'wb') as f:
    pickle.dump(model, f)
# カレントディレクトリにranknet_modelモデルを保存
with open('pkl_registory/ranknet_model.pkl', 'wb') as f:
    pickle.dump(ranknet_model, f)
# カレントディレクトリにpairwise_modelモデルを保存
with open('pkl_registory/pairwise_model.pkl', 'wb') as f:
    pickle.dump(pairwise_model, f)

In [ ]:
def find_candidates_for_second_inference(df, N):
    # 3つのモデルが同じ順位を予測するか、3つのモデルのうち2つのモデルがN位と予測し、かつ予測スコア(lambdarank)が10以上のデータをNとして返す
    candidates = df[
        ((df[f'予測順位(lambdarank)'] == N) & (df[f'予測順位(RankNet)'] == N) & (df[f'予測順位(Pairwise)'] == N)) |
        (((df[f'予測順位(lambdarank)'] == N) & (df[f'予測順位(RankNet)'] == N)) |
         ((df[f'予測順位(lambdarank)'] == N) & (df[f'予測順位(Pairwise)'] == N)) |
         ((df[f'予測順位(RankNet)'] == N) & (df[f'予測順位(Pairwise)'] == N))) &
        (df[f'予測スコア(lambdarank)'] >= 10)
    ]
    
    # 予測結果をNに設定
    df[f'予測結果({N})'] = 0
    df.loc[candidates.index, f'予測結果({N})'] = N
    
    return df

def find_candidates_for_first(df, N):
    # 3つのモデルのうち2つのモデルがN位と予測し、かつ予測スコア(lambdarank)が10以上のデータをNとして返す
    candidates_two_models = df[
        (((df[f'予測順位(lambdarank)'] == N) & (df[f'予測順位(RankNet)'] == N)) |
         ((df[f'予測順位(lambdarank)'] == N) & (df[f'予測順位(Pairwise)'] == N)) |
         ((df[f'予測順位(RankNet)'] == N) & (df[f'予測順位(Pairwise)'] == N)))
    ]
    
    # 予測結果をNに設定
    df[f'予測結果({N})'] = 0
    df.loc[candidates_two_models.index, f'予測結果({N})'] = N
    
    return df

def print_first_place_info(data, score_column, rank_column):
        # 1位のスコアを取得
        second_place = data[data[rank_column] == 1]
        
        if not second_place.empty:
            for index, row in second_place.iterrows():
                umaban = row['horse_id']
                score = row[score_column]
                rank = row[rank_column]
                result = row['result'] if 'result' in row else 'N/A'
                print(f"Umaban: {umaban}, Score: {score}, Rank: {rank}, Result: {result}")
        else:
            print("No second place found.")

In [ ]:
from pathlib import Path
import pandas as pd
import tqdm
from utils.utils_inference import *
from utils.utils import *
import os

# 青色で出力
# "RankNetの正答率が高い"
print("\033[34m二つのモデルで1位が出てくるときはオッズがつくならワイドあり\033[0m")
# csvデータを1つずつ読み出し推論
p = Path(inference_data_path)
files = list(p.glob("*inference*.csv"))
print(p)
print(files)
count = 0
for file in tqdm.tqdm(files):
    # 読み込んだファイル名を出力
    # print("=======予測ファイル名======== {}".format(file))
    print("=======予測ファイル名========\n {}".format(file))
    inference_data = pd.read_csv(file, encoding='utf-8')
    # データの確認
    # print(inference_data.info())

    # Unnamed: 0,class_list_in_raceを削除
    # inference_data = inference_data.drop(['Unnamed: 0',"class_list_in_race","date"], axis=1)
    # inference_data.info()
    inference_data['連対率'] = inference_data['連対率'].apply(
        lambda x: float(x.replace('％', '')) if isinstance(x, str) and x.strip() not in ['％', '-'] else 0.0
    )

    # '勝率'列の値を処理し、全角の'％'を半角の'%'に置き換えてからfloatに変換
    inference_data['勝率'] = inference_data['勝率'].apply(
        lambda x: float(x.replace('％', '')) if isinstance(x, str) and x.strip() not in ['％', '-'] else 0.0
    )
    
    # '複勝率'列の値を処理し、全角の'％'を半角の'%'に置き換えてからfloatに変換
    inference_data['複勝率'] = inference_data['複勝率'].apply(
        lambda x: float(x.replace('％', '')) if isinstance(x, str) and x.strip() not in ['％', '-'] else 0.0
    )
    #　=================real dataで推論=============================
    inference_data, inference_data_odds, inference_data_race_id,inference_data_answer = preprocess_for_inference(inference_data)
    # オッズの差分を計算して新しい特徴量として追加
    inference_data = add_odds_differences_for_inference(inference_data)
    

    # X_sorted_train_data.columnsの日本語文字列の半角、全角スペースを削除する
    # X_sorted_train_data.columns = X_sorted_train_data.columns.str.replace(' ', '')
    # X_sorted_train_data.columns = X_sorted_train_data.columns.str.replace('　', '')
    

    # inference_dataの特長量を設定する
    inference_data_for_predict = inference_data[X_sorted_train_data.columns]

    # ラベルエンコーディングは前処理終了後
    # inference_dataの特長量のobject型は、LabelEncoderで数値に変換する
    # object型の特長量を確認する
    object_columns = inference_data_for_predict.select_dtypes(include='object').columns
    object_columns
    for column in object_columns:
        le = LabelEncoder()
        # object型は別のラベル名にてラベルエンコーディングする
        inference_data_for_predict[column] = le.fit_transform(inference_data_for_predict[column])

    # 対数変換実施
    # inference_data = log_transform(inference_data)

    # 標準化実施
    # inference_data = standard_scaler(inference_data)

    # モデル予測
    prediction_inference_ranking = model.predict(inference_data_for_predict, num_iteration=model.best_iteration_)
    prediction_inference_ranking_ranknet = ranknet_model.predict(inference_data_for_predict)
    prediction_inference_ranking_pairwise = pairwise_model.predict(inference_data_for_predict)

    inference_data["予測スコア(lambdarank)"] = prediction_inference_ranking
    inference_data["予測スコア(RankNet)"] = prediction_inference_ranking_ranknet
    inference_data["予測スコア(Pairwise)"] = prediction_inference_ranking_pairwise
    inference_data["goal_number"] = inference_data_answer
    # 2位予測用にデータを保存
    inference_data.to_csv(file, index=False)

    # シンプルに予測結果の平均を取る
    prediction_inference = (prediction_inference_ranking + prediction_inference_ranking_ranknet + prediction_inference_ranking_pairwise) / 3
    # データフレームを作成
    df_prediction_test_ranking = pd.DataFrame({
        "horse_id": inference_data["umaban"],
        "予測スコア(lambdarank)": prediction_inference_ranking,
        "予測スコア(RankNet)": prediction_inference_ranking_ranknet,
        "予測スコア(Pairwise)": prediction_inference_ranking_pairwise,
        "predict_amsamble": prediction_inference,
        "odds": inference_data_odds,
        "goal": inference_data_answer,
        "race_id": inference_data_race_id,
        "distance": inference_data["distance"],
        "condition": inference_data["condition"],
        "weather": inference_data["weather"],
        "ground_state": inference_data["ground_state"],
    })

    # 予測スコアの高い順にソート
    df_prediction_test_ranking = df_prediction_test_ranking.sort_values(by="予測スコア(lambdarank)", ascending=False)
    # 予測順位を計算
    df_prediction_test_ranking["予測順位(lambdarank)"] = df_prediction_test_ranking["予測スコア(lambdarank)"].rank(ascending=False, method='first').astype(int)
    # 予測スコアの高い順にソート
    df_prediction_test_ranking = df_prediction_test_ranking.sort_values(by="予測スコア(RankNet)", ascending=False)
    # 予測順位を計算
    df_prediction_test_ranking["予測順位(RankNet)"] = df_prediction_test_ranking["予測スコア(RankNet)"].rank(ascending=False, method='first').astype(int)
    # 予測スコアの高い順にソート
    df_prediction_test_ranking = df_prediction_test_ranking.sort_values(by="予測スコア(Pairwise)", ascending=False)
    # 予測順位を計算
    df_prediction_test_ranking["予測順位(Pairwise)"] = df_prediction_test_ranking["予測スコア(Pairwise)"].rank(ascending=False, method='first').astype(int)
    # 予測スコアの高い順にソート
    df_prediction_test_ranking = df_prediction_test_ranking.sort_values(by="predict_amsamble", ascending=False)
    # 予測順位を計算
    df_prediction_test_ranking["予測順位(amsamble)"] = df_prediction_test_ranking["predict_amsamble"].rank(ascending=False, method='first').astype(int)
    # インデックスをリセット
    df_prediction_test_ranking = df_prediction_test_ranking.reset_index(drop=True)
    # 最後に着順にソート
    df_prediction_test_ranking = df_prediction_test_ranking.sort_values(by="goal")

    # 結果を確認
    # print(df_prediction_test_ranking.head())
    # pandasをcsvに保存
    # inference_data_path + /inference_resultにディレクトリを作成し、csvを保存する
    inference_data_path_result = inference_data_path + "inference_result/"
    if not os.path.isdir(inference_data_path_result):
        os.mkdir(inference_data_path_result)
    file_name = file.name
    saving_path = inference_data_path_result + "prediction_" +  str(file_name) + "_.csv"
    # print("saving_path:{}".format(saving_path))
    df_prediction_test_ranking.to_csv(saving_path, index=False)

    # 推論データに対して予測を行う。1位と2位のスコアの差を計算し、閾値を適用して各モデルごとに1位を予測する
    df_prediction_test_ranking_ = prediction_print(df_prediction_test_ranking,optimal_threshold_lambdarank,optimal_threshold_ranknet,optimal_threshold_pairwise)
    print("Candidate from multiple models")
    find_candidates_for_first(df_prediction_test_ranking,1)
    print_first_place_info(df_prediction_test_ranking, "予測スコア(RankNet)", "予測結果(1)")
    # =================================================================
    # 予測結果がシビアすぎて全然買い目が出ないので過去の情報も一応出力しておく
    # =================================================================
    # すべてのモデルのスコアが正で1位かつスコアが0以上のものをフィルタリング
    # utils_inference.add_prediction_info(df_prediction_test_ranking)

    count +=1


In [ ]:
# 2位の予測モデルの生成
# 各モデルのスコアを計算する関数
def add_model_scores(data, model, model_name):
    scores = model.predict(data[X_sorted_train_data.columns])
    data[f'pred_score_{model_name}'] = scores
    return data

# 学習データにスコアを追加
# X_sorted_train_data_copy = add_model_scores(X_sorted_train_data_copy, model, 'lambdarank')
# X_sorted_train_data_copy = add_model_scores(X_sorted_train_data_copy, ranknet_model, 'RankNet')
X_sorted_train_data_copy = add_model_scores(X_sorted_train_data_copy, pairwise_model, 'Pairwise')

# 検証データにスコアを追加
# X_sorted_valid_data_copy = add_model_scores(X_sorted_valid_data_copy, model, 'lambdarank')
# X_sorted_valid_data_copy = add_model_scores(X_sorted_valid_data_copy, ranknet_model, 'RankNet')
X_sorted_valid_data_copy = add_model_scores(X_sorted_valid_data_copy, pairwise_model, 'Pairwise')

# テストデータにスコアを追加
# test_ranking_copy = add_model_scores(test_ranking_copy, model, 'lambdarank')
# test_ranking_copy = add_model_scores(test_ranking_copy, ranknet_model, 'RankNet')
test_ranking_copy = add_model_scores(test_ranking_copy, pairwise_model, 'Pairwise')

def add_race_features(data, model_name, race_id=True):
    if race_id:
        # 学習データでrace_idがクエリになるようにしている
        groupby_obj = data.groupby("race_id")
    else:
        # 推論データでrace_idがない場合は全体をグループ化
        groupby_obj = data

    if race_id:
        # 1位のスコア
        top_scores = groupby_obj[f'pred_score_{model_name}'].transform('max')
        data[f'top_score_{model_name}'] = top_scores
        
        # 各馬のスコアと1位のスコアの差分
        data[f'score_diff_{model_name}'] = top_scores - data[f'pred_score_{model_name}']
        
        # 1位と2位のスコアの差分
        second_scores = groupby_obj[f'pred_score_{model_name}'].transform(lambda x: x.nlargest(2).iloc[-1])
        data[f'top2_score_diff_{model_name}'] = top_scores - second_scores
        
        # レース内でのスコアの標準偏差
        data[f'score_std_{model_name}'] = groupby_obj[f'pred_score_{model_name}'].transform('std')
        
        # レース内でのスコアの最大-最小
        data[f'score_range_{model_name}'] = groupby_obj[f'pred_score_{model_name}'].transform(lambda x: x.max() - x.min())
        
        # 1位のスコアと平均スコアの差
        mean_scores = groupby_obj[f'pred_score_{model_name}'].transform('mean')
        data[f'top_mean_score_diff_{model_name}'] = top_scores - mean_scores
        
        # 上位3頭のスコアの平均
        top3_mean_scores = groupby_obj[f'pred_score_{model_name}'].transform(lambda x: x.nlargest(3).mean())
        data[f'top3_mean_score_{model_name}'] = top3_mean_scores
        
        # 2位以下のスコアの平均
        below2_mean_scores = groupby_obj[f'pred_score_{model_name}'].transform(lambda x: x.nsmallest(len(x)-1).mean())
        data[f'below2_mean_score_{model_name}'] = below2_mean_scores
        
        # 2位以下のスコアの標準偏差
        below2_std_scores = groupby_obj[f'pred_score_{model_name}'].transform(lambda x: x.nsmallest(len(x)-1).std())
        data[f'below2_score_std_{model_name}'] = below2_std_scores
        
        # 2位と3位のスコア差
        third_scores = groupby_obj[f'pred_score_{model_name}'].transform(lambda x: x.nlargest(3).iloc[-1])
        data[f'second_third_score_diff_{model_name}'] = second_scores - third_scores
        
        # 2位と最下位のスコア差
        last_scores = groupby_obj[f'pred_score_{model_name}'].transform('min')
        data[f'second_last_score_diff_{model_name}'] = second_scores - last_scores
    else:
        # 1位のスコア
        top_score = data[f'pred_score_{model_name}'].max()
        data[f'top_score_{model_name}'] = top_score
        
        # 各馬のスコアと1位のスコアの差分
        data[f'score_diff_{model_name}'] = top_score - data[f'pred_score_{model_name}']
        
        # 1位と2位のスコアの差分
        second_score = data[f'pred_score_{model_name}'].nlargest(2).iloc[-1]
        data[f'top2_score_diff_{model_name}'] = top_score - second_score
        
        # スコアの標準偏差
        score_std = data[f'pred_score_{model_name}'].std()
        data[f'score_std_{model_name}'] = score_std
        
        # スコアの最大-最小
        score_range = data[f'pred_score_{model_name}'].max() - data[f'pred_score_{model_name}'].min()
        data[f'score_range_{model_name}'] = score_range
        
        # 1位のスコアと平均スコアの差
        mean_score = data[f'pred_score_{model_name}'].mean()
        data[f'top_mean_score_diff_{model_name}'] = top_score - mean_score
        
        # 上位3頭のスコアの平均
        top3_mean_score = data[f'pred_score_{model_name}'].nlargest(3).mean()
        data[f'top3_mean_score_{model_name}'] = top3_mean_score
        
        # 2位以下のスコアの平均
        below2_mean_score = data[f'pred_score_{model_name}'].nsmallest(len(data)-1).mean()
        data[f'below2_mean_score_{model_name}'] = below2_mean_score
        
        # 2位以下のスコアの標準偏差
        below2_std_score = data[f'pred_score_{model_name}'].nsmallest(len(data)-1).std()
        data[f'below2_score_std_{model_name}'] = below2_std_score
        
        # 2位と3位のスコア差
        third_score = data[f'pred_score_{model_name}'].nlargest(3).iloc[-1]
        data[f'second_third_score_diff_{model_name}'] = second_score - third_score
        
        # 2位と最下位のスコア差
        last_score = data[f'pred_score_{model_name}'].min()
        data[f'second_last_score_diff_{model_name}'] = second_score - last_score
    
    return data

# 学習データに特長量を追加
# X_sorted_train_data_copy = add_race_features(X_sorted_train_data_copy, 'lambdarank')
# X_sorted_train_data_copy = add_race_features(X_sorted_train_data_copy, 'RankNet')
X_sorted_train_data_copy = add_race_features(X_sorted_train_data_copy, 'Pairwise')

# 検証データに特長量を追加
# X_sorted_valid_data_copy = add_race_features(X_sorted_valid_data_copy, 'lambdarank')
# X_sorted_valid_data_copy = add_race_features(X_sorted_valid_data_copy, 'RankNet')
X_sorted_valid_data_copy = add_race_features(X_sorted_valid_data_copy, 'Pairwise')

# テストデータに特長量を追加
# test_ranking_copy = add_race_features(test_ranking_copy, 'lambdarank')
# test_ranking_copy = add_race_features(test_ranking_copy, 'RankNet')
test_ranking_copy = add_race_features(test_ranking_copy, 'Pairwise')

print(test_ranking_copy.columns)



# pred_score_Pairwiseの逆数、対数化、標準化、正規化を行って性能改善するかを試す
# X_sorted_train_data_copy[f'pred_score_Pairwise_inv'] = 1 / X_sorted_train_data_copy[f'pred_score_Pairwise']
# X_sorted_valid_data_copy[f'pred_score_Pairwise_inv'] = 1 / X_sorted_valid_data_copy[f'pred_score_Pairwise']
# test_ranking_copy[f'pred_score_Pairwise_inv'] = 1 / test_ranking_copy[f'pred_score_Pairwise']
# X_sorted_train_data_copy[f'pred_score_Pairwise_log'] = np.log1p(X_sorted_train_data_copy[f'pred_score_Pairwise'])
# X_sorted_valid_data_copy[f'pred_score_Pairwise_log'] = np.log1p(X_sorted_valid_data_copy[f'pred_score_Pairwise'])
# test_ranking_copy[f'pred_score_Pairwise_log'] = np.log1p(test_ranking_copy[f'pred_score_Pairwise'])
# X_sorted_train_data_copy[f'pred_score_Pairwise_std'] = (X_sorted_train_data_copy[f'pred_score_Pairwise'] - X_sorted_train_data_copy[f'pred_score_Pairwise'].mean()) / X_sorted_train_data_copy[f'pred_score_Pairwise'].std()
# X_sorted_valid_data_copy[f'pred_score_Pairwise_std'] = (X_sorted_valid_data_copy[f'pred_score_Pairwise'] - X_sorted_valid_data_copy[f'pred_score_Pairwise'].mean()) / X_sorted_valid_data_copy[f'pred_score_Pairwise'].std()
# test_ranking_copy[f'pred_score_Pairwise_std'] = (test_ranking_copy[f'pred_score_Pairwise'] - test_ranking_copy[f'pred_score_Pairwise'].mean()) / test_ranking_copy[f'pred_score_Pairwise'].std()
# X_sorted_train_data_copy[f'pred_score_Pairwise_norm'] = (X_sorted_train_data_copy[f'pred_score_Pairwise'] - X_sorted_train_data_copy[f'pred_score_Pairwise'].min()) / (X_sorted_train_data_copy[f'pred_score_Pairwise'].max() - X_sorted_train_data_copy[f'pred_score_Pairwise'].min())
# X_sorted_valid_data_copy[f'pred_score_Pairwise_norm'] = (X_sorted_valid_data_copy[f'pred_score_Pairwise'] - X_sorted_valid_data_copy[f'pred_score_Pairwise'].min()) / (X_sorted_valid_data_copy[f'pred_score_Pairwise'].max() - X_sorted_valid_data_copy[f'pred_score_Pairwise'].min())
# test_ranking_copy[f'pred_score_Pairwise_norm'] = (test_ranking_copy[f'pred_score_Pairwise'] - test_ranking_copy[f'pred_score_Pairwise'].min()) / (test_ranking_copy[f'pred_score_Pairwise'].max() - test_ranking_copy[f'pred_score_Pairwise'].min())

# pred_score_Pairwiseの特長量寄与度が高いのでいったんなしでどうなるか見てみる-> pairwiseのスコアとっても上がりました。
X_sorted_train_data_copy.drop([f'pred_score_Pairwise'], axis=1, inplace=True)
X_sorted_valid_data_copy.drop([f'pred_score_Pairwise'], axis=1, inplace=True)
test_ranking_copy.drop([f'pred_score_Pairwise'], axis=1, inplace=True)


In [ ]:
# 着順1位の馬を削除
# 学習データから1位の馬を削除
def remove_top_horse(data):
    # 着順が1位の馬を削除
    data = data[data['goal_number'] != 1]
    return data
X_sorted_train_data_copy = remove_top_horse(X_sorted_train_data_copy)

# 検証データから1位の馬を削除
X_sorted_valid_data_copy = remove_top_horse(X_sorted_valid_data_copy)

# テストデータから1位の馬を削除
test_ranking_copy = remove_top_horse(test_ranking_copy)
test_ranking_race_id = test_ranking_copy["race_id"]


In [ ]:
# 目的関数であるgoal_number_replaceを振りなおす。
# まずは1位の予測と同じ
X_sorted_train_data_copy['goal_number_replace'] = X_sorted_train_data_copy['goal_number'].apply(
    lambda x: 12 if x == 2 else (1 if x == 3 else 0))
X_sorted_valid_data_copy['goal_number_replace'] = X_sorted_valid_data_copy['goal_number'].apply(
    lambda x: 12 if x == 2 else (1 if x == 3 else 0))
test_ranking_copy['goal_number_replace'] = test_ranking_copy['goal_number'].apply(
    lambda x: 12 if x == 2 else (1 if x == 3 else 0))

In [ ]:
# 1位の馬がなくなったのでクエリを振りなおし
# trainデータのクエリを作成
from utils.utils import create_query
train_query, X_sorted_train_data_copy = create_query(X_sorted_train_data_copy,"train")
# validデータのクエリを作成
valid_query, X_sorted_valid_data_copy = create_query(X_sorted_valid_data_copy,"valid")


In [ ]:
# 1========================================================
y_train_ranking = X_sorted_train_data_copy['goal_number_replace']
y_valid_ranking = X_sorted_valid_data_copy['goal_number_replace']
y_test_true_ranking = test_ranking_copy['goal_number_replace']
# 2========================================================
y_train_ranking_goal = X_sorted_train_data_copy['goal_number']
y_valid_ranking_goal = X_sorted_valid_data_copy['goal_number']
y_test_true_ranking_goal = test_ranking_copy['goal_number']
# 3========================================================
X_sorted_train_data_copy = X_sorted_train_data_copy.drop('goal_number', axis=1)
X_sorted_valid_data_copy = X_sorted_valid_data_copy.drop('goal_number', axis=1)
test_ranking_copy = test_ranking_copy.drop('goal_number', axis=1)
# 4========================================================
X_sorted_train_data_copy = X_sorted_train_data_copy.drop('goal_number_replace', axis=1)
X_sorted_valid_data_copy = X_sorted_valid_data_copy.drop('goal_number_replace', axis=1)
test_ranking_copy = test_ranking_copy.drop('goal_number_replace', axis=1)
# 5========================================================
X_sorted_train_data_copy = X_sorted_train_data_copy.drop('race_id', axis=1)
X_sorted_valid_data_copy = X_sorted_valid_data_copy.drop('race_id', axis=1)
test_ranking_copy = test_ranking_copy.drop('race_id', axis=1)

In [ ]:
# スクレイピングするごとにカラム名が変化してくので、カラム名のスペースを削除する処理を学習前に追加
# X_sorted_train_data.columnsの日本語文字列の半角、全角スペースを削除する
X_sorted_train_data_copy.columns = X_sorted_train_data_copy.columns.str.replace(' ', '')
X_sorted_train_data_copy.columns = X_sorted_train_data_copy.columns.str.replace('　', '')

# X_sorted_valid_data.columnsの日本語文字列の半角、全角スペースを削除する
X_sorted_valid_data_copy.columns = X_sorted_valid_data_copy.columns.str.replace(' ', '')
X_sorted_valid_data_copy.columns = X_sorted_valid_data_copy.columns.str.replace('　', '')

# test_ranking.columnsの日本語文字列の半角、全角スペースを削除する
test_ranking_copy.columns = test_ranking_copy.columns.str.replace(' ', '')
test_ranking_copy.columns = test_ranking_copy.columns.str.replace('　', '')

# データ確認用にX_sorted_train_data_copyを保存
X_sorted_train_data_copy.to_csv("X_sorted_train_data_copy.csv", index=False)


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
from sklearn.metrics import ndcg_score

# データの準備
# X_sorted_train_data, y_train_ranking, train_query
# X_sorted_valid_data, y_valid_ranking, valid_query

# ランキング学習
lambdarank_model_second = lgb.LGBMRanker(
    random_state=42,
    objective='lambdarank',
    metric='ndcg',
    n_estimators=500,              # 決定木の個数(default:100) 5000のほうが10000より高い
    learning_rate=0.1,            # 学習率(default:0.1)
    num_leaves=10,                 # 決定木にある分岐の個数(default:31)
    max_depth=10,                  # 決定木の深さの最大値(default:-1)
    min_child_samples=40,         # 決定木のノードに含まれる最小サンプル数(default:20)
    # feature_fraction=0.8,
    # bagging_fraction=0.8,
    # bagging_freq=10,
    # lambda_l1=0.1,
    # lambda_l2=0.1,
    # min_split_gain=0.1,
    # min_child_weight=0.1
    # rambdalank_truncation_level=10,
    max_position=3, # 3位までの順位を予測する,
    # lambdarank_truncation_level = 5,
)

# モデルの学習
lambdarank_model_second.fit(
    X_sorted_train_data_copy,
    y_train_ranking,
    group=train_query,
    eval_set=[(X_sorted_valid_data_copy, y_valid_ranking)],
    eval_group=[valid_query],
    eval_at=[1, 2, 3],
    eval_metric='ndcg',
)

In [ ]:
# トレーニング結果の取得
evals_result = lambdarank_model_second.evals_result_

# NDCGスコアのプロット
# NDCGスコアは1になるほど予測モデルの性能が良いことを占める。1,3,5は上位何件の結果を反映するか。
plt.figure(figsize=(10, 6))
plt.plot(evals_result['valid_0']['ndcg@1'], label='NDCG@1')
plt.plot(evals_result['valid_0']['ndcg@2'], label='NDCG@2')
plt.plot(evals_result['valid_0']['ndcg@3'], label='NDCG@3')
plt.xlabel('Iterations')
plt.ylabel('NDCG Score')
plt.title('NDCG Score over Iterations')
plt.legend()
plt.grid(True)
plt.show()
# model.best_iteration_のNDCGを数値で出力
print("Best NDCG@1: {:.4f} in iteration: {}".format(evals_result['valid_0']['ndcg@1'][lambdarank_model_second.best_iteration_ - 1], lambdarank_model_second.best_iteration_))
print("Best NDCG@2: {:.4f} in iteration: {}".format(evals_result['valid_0']['ndcg@2'][lambdarank_model_second.best_iteration_ - 1], lambdarank_model_second.best_iteration_))
print("Best NDCG@3: {:.4f} in iteration: {}".format(evals_result['valid_0']['ndcg@3'][lambdarank_model_second.best_iteration_ - 1], lambdarank_model_second.best_iteration_))

In [ ]:
# アンサンブル候補①LightGBM(RankNet)

# LightGBMデータセット
train_data = lgb.Dataset(X_sorted_train_data_copy, label=y_train_ranking, group=train_query)
valid_data = lgb.Dataset(X_sorted_valid_data_copy, label=y_valid_ranking, group=valid_query)
# パラメータの設定
params = {
    'objective': 'rank_xendcg',
    'metric': 'ndcg',
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l1': 0.1,
    'lambda_l2': 0.2,
    'ndcg_at': [1, 2, 3],
    'n_estimators':500,              # 決定木の個数(default:100) 5000のほうが10000より高い
    'learning_rate':0.05,            # 学習率(default:0.1)
    'max_depth':8,                  # 決定木の深さの最大値(default:-1)
    "min_data_in_leaf":40,         # 決定木のノードに含まれる最小サンプル数(default:20)
    "lambdarank_truncation_level":1,
}

evaluation_results = {}
# モデルの学習
ranknet_model_second = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    callbacks=[lgb.record_evaluation(evaluation_results)],
)

# 学習結果の可視化をndcgで行う
# NDCGスコアのプロット
# NDCGスコアは1になるほど予測モデルの性能が良いことを占める。1,3,5は上位何件の結果を反映するか。
plt.figure(figsize=(10, 6))
plt.plot(evaluation_results['valid_0']['ndcg@1'], label='NDCG@1')
plt.plot(evaluation_results['valid_0']['ndcg@2'], label='NDCG@2')
plt.plot(evaluation_results['valid_0']['ndcg@3'], label='NDCG@3')
plt.xlabel('Iterations')
plt.ylabel('NDCG Score')
plt.title('NDCG Score over Iterations')
plt.legend()
plt.grid(True)
plt.show()
# model.best_iteration_のNDCGを数値で出力
print("Best NDCG@1: {:.4f} in iteration: {}".format(ranknet_model_second.best_score['valid_0']['ndcg@1'], ranknet_model_second.best_iteration))
# 2も出力
print("Best NDCG@2: {:.4f} in iteration: {}".format(ranknet_model_second.best_score['valid_0']['ndcg@2'], ranknet_model_second.best_iteration))
# 3も出力
print("Best NDCG@3: {:.4f} in iteration: {}".format(ranknet_model_second.best_score['valid_0']['ndcg@3'], ranknet_model_second.best_iteration))



In [ ]:
# アンサンブル候補②XGBoost(Pairwise)
from xgboost import XGBRanker
# モデルの定義
pairwise_model_second = XGBRanker(
    objective='rank:pairwise',
    eval_metric='ndcg',
    n_estimators=1000,              # 決定木の個数(default:100) 5000のほうが10000より高い
    learning_rate=0.1,            # 学習率(default:0.1)
    max_depth=10,                    # 決定木の深さの最大値(default:6)
    # subsample=0.8,                  # データのサンプリング比率(default:1)
    # colsample_bytree=0.8,           # 列のサンプリング比率(default:1)
    # min_child_weight=0.5,           # 葉の重みの最小値(default:1)
    # reg_lambda=1.0,                 # L2正則化の強さ(default:1)
    # gamma=0.1,                      # 葉の追加分岐を行うかの閾値(default:0)
    # n_jobs=-1,                      # 並列処理の数(default:1)
    random_state=42,                # 乱数のシード値(default:0)
    max_position=3, # 3位までの順位を予測する
    min_child_samples=30,         # 決定木のノードに含まれる最小サンプル数(default:20)
    # lambdarank_truncation_level=5,
)
# モデルの学習
pairwise_model_second.fit(
    X_sorted_train_data_copy,
    y_train_ranking,
    group=train_query,
    eval_set=[(X_sorted_valid_data_copy, y_valid_ranking)],
    eval_group=[valid_query],
    verbose=100,
    early_stopping_rounds=100,
)

# 学習結果の可視化をndcgで行う
# NDCGスコアのプロット
# NDCGスコアは1になるほど予測モデルの性能が良いことを占める。1,3,5は上位何件の結果を反映するか。
plt.figure(figsize=(10, 6))
plt.plot(pairwise_model_second.evals_result()['validation_0']['ndcg'], label='NDCG')
plt.xlabel('Iterations')
plt.ylabel('NDCG Score')
plt.title('NDCG Score over Iterations')
plt.legend()
plt.grid(True)
plt.show()
# model.best_iteration_のNDCGを数値で出力
print("Best NDCG: {:.4f} in iteration: {}".format(pairwise_model_second.best_score, pairwise_model_second.best_iteration))

In [ ]:
#　テストデータで推論
# LighGBM(lambdarank)の推論
prediction_test_ranking = lambdarank_model_second.predict(test_ranking_copy, num_iteration=lambdarank_model_second.best_iteration_)
# LightGBM(RankNet)の推論
prediction_test_ranking_ranknet = ranknet_model_second.predict(test_ranking_copy)
# XGBoost(Pairwise)の推論
prediction_test_ranking_pairwise = pairwise_model_second.predict(test_ranking_copy)
# 上記の平均をアンサンブルモデルとする
prediction_test_ranking_ensemble = (prediction_test_ranking + prediction_test_ranking_ranknet + prediction_test_ranking_pairwise) / 3

df_prediction_test_ranking = pd.DataFrame({
    "馬番号": test_ranking_copy["umaban"],
    "予測スコア(lambdarank)": prediction_test_ranking,
    "予測スコア(RankNet)": prediction_test_ranking_ranknet,
    "予測スコア(Pairwise)": prediction_test_ranking_pairwise,
    "ensemble_prediction": prediction_test_ranking_ensemble,
    "着順関連度": y_test_true_ranking,
    "着順": y_test_true_ranking_goal,
    "race_id": test_ranking_race_id,
    "distance": test_ranking_copy["distance"],
    "ground_state": test_ranking_copy["ground_state"],
    "weather": test_ranking_copy["weather"],
    "condition": test_ranking_copy["condition"],
})


# 同じレースidのうちスコアの大きい順に1位から予測順位を計算する
def rank_predictions(data, score_column, rank_column):
    data[rank_column] = data.groupby("race_id")[score_column].rank(ascending=False, method='first')
    data[rank_column] = data.groupby("race_id")[rank_column].transform(lambda x: x + 1)
    return data
# 予測順位をつける
df_prediction_test_ranking = rank_predictions(df_prediction_test_ranking, "予測スコア(lambdarank)", "予測順位(lambdarank)")
df_prediction_test_ranking = rank_predictions(df_prediction_test_ranking, "予測スコア(RankNet)", "予測順位(RankNet)")
df_prediction_test_ranking = rank_predictions(df_prediction_test_ranking, "予測スコア(Pairwise)", "予測順位(Pairwise)")
df_prediction_test_ranking = rank_predictions(df_prediction_test_ranking, "ensemble_prediction", "ensemble_prediction_rank")

df_prediction_test_ranking["予測順位(lambdarank)"] = df_prediction_test_ranking["予測順位(lambdarank)"].astype(int)
df_prediction_test_ranking["予測順位(RankNet)"] = df_prediction_test_ranking["予測順位(RankNet)"].astype(int)
df_prediction_test_ranking["予測順位(Pairwise)"] = df_prediction_test_ranking["予測順位(Pairwise)"].astype(int)
df_prediction_test_ranking["ensemble_prediction_rank"] = df_prediction_test_ranking["ensemble_prediction_rank"].astype(int)

# 3つのモデルがすべて1位と判断したときだけ1位とするアンサンブルモデル
# df_prediction_test_ranking['ensemble_prediction'] = (
#     (df_prediction_test_ranking['予測順位(lambdarank)'] == 1) &
#     (df_prediction_test_ranking['予測順位(RankNet)'] == 1) &
#     (df_prediction_test_ranking['予測順位(Pairwise)'] == 1)
# ).astype(int)

# 予測スコアの分布を可視化
plt.figure(figsize=(10, 6))

# XGBoost(Pairwise)のヒストグラム
plt.hist(prediction_test_ranking_pairwise, bins=1, alpha=0.3, label='XGBoost(Pairwise)', color='blue')

# LightGBM(lambdarank)のヒストグラム
plt.hist(prediction_test_ranking, bins=50, alpha=0.3, label='LightGBM(lambdarank)', color='green')

# LightGBM(RankNet)のヒストグラム
plt.hist(prediction_test_ranking_ranknet, bins=50, alpha=0.3, label='LightGBM(RankNet)', color='red')

plt.xlabel('Predicted Score')
plt.ylabel('Frequency')
plt.title('Predicted Score Distribution')
plt.legend()
plt.grid(True)
plt.show()

# 予測結果を保存
df_prediction_test_ranking.to_csv("prediction_test_secandary_ranking.csv", index=False)
print(prediction_test_ranking_pairwise.mean())



In [ ]:
# 2位のスコアにおける予測結果の評価
print("LightGBM(lambdarank)の評価")
rank_evaluation_N(df_prediction_test_ranking, '予測順位(lambdarank)')
print("LightGBM(RankNet)の評価")
rank_evaluation_N(df_prediction_test_ranking, '予測順位(RankNet)')
print("XGBoost(Pairwise)の評価")
rank_evaluation_N(df_prediction_test_ranking, '予測順位(Pairwise)')
print("アンサンブルモデルの評価")
rank_evaluation_N(df_prediction_test_ranking, 'ensemble_prediction_rank')

In [ ]:
# 同じようなアプローチで2位と3位のスコアの差から2位の予測精度の向上にトライ
# 1位と2位のスコアの差を可視化する→閾値を決定してコンフュージョンマトリックスで正解率を確認する
df_prediction_test_ranking['score_diff_lambdarank'] = calculate_score_diff(df_prediction_test_ranking, 'lambdarank')
df_prediction_test_ranking['score_diff_RankNet'] = calculate_score_diff(df_prediction_test_ranking, 'RankNet')
df_prediction_test_ranking['score_diff_Pairwise'] = calculate_score_diff(df_prediction_test_ranking, 'Pairwise')

plot_score_diff(df_prediction_test_ranking, 'lambdarank', 2)
plot_score_diff(df_prediction_test_ranking, 'RankNet', 2)
plot_score_diff(df_prediction_test_ranking, 'Pairwise', 2)

optimal_threshold_lambdarank, optimal_threshold_pr_lambdarank = find_optimal_threshold(df_prediction_test_ranking, 'lambdarank', 2)
optimal_threshold_ranknet, optimal_threshold_pr_ranknet = find_optimal_threshold(df_prediction_test_ranking, 'RankNet', 2)
optimal_threshold_pairwise, optimal_threshold_pr_pairwise = find_optimal_threshold(df_prediction_test_ranking, 'Pairwise', 2)
print(f'Optimal Threshold (ROC) for lambdarank: {optimal_threshold_lambdarank}')
print(f'Optimal Threshold (Precision-Recall) for lambdarank: {optimal_threshold_pr_lambdarank}')
print(f'Optimal Threshold (ROC) for RankNet: {optimal_threshold_ranknet}')
print(f'Optimal Threshold (Precision-Recall) for RankNet: {optimal_threshold_pr_ranknet}')
print(f'Optimal Threshold (ROC) for Pairwise: {optimal_threshold_pairwise}')
print(f'Optimal Threshold (Precision-Recall) for Pairwise: {optimal_threshold_pr_pairwise}')

df_prediction_test_ranking = apply_threshold_second(df_prediction_test_ranking, 'lambdarank', optimal_threshold_lambdarank)
df_prediction_test_ranking = apply_threshold_second(df_prediction_test_ranking, 'RankNet', optimal_threshold_ranknet)
df_prediction_test_ranking = apply_threshold_second(df_prediction_test_ranking, 'Pairwise', optimal_threshold_pairwise)

print("LightGBM(lambdarank)の評価")
print(df_prediction_test_ranking['予測結果(lambdarank)'])
rank_evaluation_N(df_prediction_test_ranking, '予測結果(lambdarank)')
print("LightGBM(RankNet)の評価")
rank_evaluation_N(df_prediction_test_ranking, '予測結果(RankNet)')
print("XGBoost(Pairwise)の評価")
rank_evaluation_N(df_prediction_test_ranking, '予測結果(Pairwise)')

In [ ]:
# 特徴重要度の可視化
df_importance = pd.DataFrame({"columns": X_sorted_train_data_copy.columns, "importance": pairwise_model_second.feature_importances_})
df_importance.sort_values("importance", ascending=False, inplace=True)
print(df_importance)

In [ ]:
from pathlib import Path
import pandas as pd
import tqdm
from utils.utils_inference import *

# 青色で出力
# "RankNetの正答率が高い"
print("\033[34m3つのモデルで1番または２つのモデルで1位かつlambdarankスコアが10以上を閾値とする\033[0m")
# csvデータを1つずつ読み出し推論
p = Path(inference_data_path)
files = list(p.glob("*inference*.csv"))
print(p)
print(files)
count = 0
for file in tqdm.tqdm(files):
    # 読み込んだファイル名を出力
    # print("=======予測ファイル名======== {}".format(file))
    print("=======予測ファイル名========\n {}".format(file))
    inference_data = pd.read_csv(file, encoding='utf-8')
    # データの確認
    # print(inference_data.info())

    # Unnamed: 0,class_list_in_raceを削除
    # inference_data = inference_data.drop(['Unnamed: 0',"class_list_in_race","date"], axis=1)

    #　=================real dataで推論=============================
    # inference_data, inference_data_odds, inference_data_race_id,inference_data_answer = preprocess_for_inference(inference_data,True)
    # オッズの差分を計算して新しい特徴量として追加
    inference_data = add_odds_differences_for_inference(inference_data)
    
    # 推論済みの特長量"予測スコア(lambdarank)"をpred_score_lambdarankに変更
    inference_data = inference_data.rename(columns={'予測スコア(lambdarank)': 'pred_score_lambdarank'})
    # 推論済みの特長量"予測スコア(RankNet)"をpred_score_RankNetに変更
    inference_data = inference_data.rename(columns={'予測スコア(RankNet)': 'pred_score_RankNet'})
    # 推論済みの特長量"予測スコア(Pairwise)"をpred_score_Pairwiseに変更
    inference_data = inference_data.rename(columns={'予測スコア(Pairwise)': 'pred_score_Pairwise'})
    # テストデータに特長量を追加
    inference_data = add_race_features(inference_data, 'lambdarank', False)
    inference_data = add_race_features(inference_data, 'RankNet', False)
    inference_data = add_race_features(inference_data, 'Pairwise', False)

    # 予測スコア1位の馬を削除
    def remove_top_horse_no_race_id(data, model_name):
        # 最大スコアの馬情報を取得
        max_score = data[f'pred_score_{model_name}'].max()
        
        # 最大スコアの馬情報を削除
        data = data[data[f'pred_score_{model_name}'] != max_score]
        
        return data
    # 推論データから最大スコアの馬情報を削除
    inference_data = remove_top_horse_no_race_id(inference_data, 'lambdarank')
    inference_data = remove_top_horse_no_race_id(inference_data, 'RankNet')
    inference_data = remove_top_horse_no_race_id(inference_data, 'Pairwise')
    inference_data.info()
    inference_data_odds = inference_data["odds"]
    # inference_dataの特長量を設定する
    inference_data = inference_data[X_sorted_train_data_copy.columns]

    # ラベルエンコーディングは前処理終了後
    # inference_dataの特長量のobject型は、LabelEncoderで数値に変換する
    # object型の特長量を確認する
    object_columns = inference_data.select_dtypes(include='object').columns
    object_columns
    for column in object_columns:
        # print(column)
        le = LabelEncoder()
        # object型は別のラベル名にてラベルエンコーディングする
        inference_data[column] = le.fit_transform(inference_data[column])
    # print(inference_data.info())

    # 対数変換実施
    # inference_data = log_transform(inference_data)

    # 標準化実施
    # inference_data = standard_scaler(inference_data)

    # モデル予測
    prediction_inference_ranking = lambdarank_model_second.predict(inference_data, num_iteration=lambdarank_model_second.best_iteration_)
    prediction_inference_ranking_ranknet = ranknet_model_second.predict(inference_data)
    prediction_inference_ranking_pairwise = pairwise_model_second.predict(inference_data)
    # シンプルに予測結果の平均を取る
    prediction_inference = (prediction_inference_ranking + prediction_inference_ranking_ranknet + prediction_inference_ranking_pairwise) / 3
    # データフレームを作成
    df_prediction_test_ranking = pd.DataFrame({
        "horse_id": inference_data["umaban"],
        "予測スコア(lambdarank)": prediction_inference_ranking,
        "予測スコア(RankNet)": prediction_inference_ranking_ranknet,
        "予測スコア(Pairwise)": prediction_inference_ranking_pairwise,
        "predict_amsamble": prediction_inference,
        "odds": inference_data_odds,
        "distance": inference_data["distance"],
        "condition": inference_data["condition"],
        "weather": inference_data["weather"],
        "ground_state": inference_data["ground_state"],
        "race_id":["test"] * len(inference_data)
    })

    # 同じレースidのうちスコアの大きい順に1位から予測順位を計算する
    def rank_predictions(data, score_column, rank_column):
        data[rank_column] = data[score_column].rank(ascending=False, method='first')
        data[rank_column] = data[rank_column].transform(lambda x: x + 1)
        return data
    # 予測順位をつける
    df_prediction_test_ranking = rank_predictions(df_prediction_test_ranking, "予測スコア(lambdarank)", "予測順位(lambdarank)")
    df_prediction_test_ranking = rank_predictions(df_prediction_test_ranking, "予測スコア(RankNet)", "予測順位(RankNet)")
    df_prediction_test_ranking = rank_predictions(df_prediction_test_ranking, "予測スコア(Pairwise)", "予測順位(Pairwise)")

    df_prediction_test_ranking["予測順位(lambdarank)"] = df_prediction_test_ranking["予測順位(lambdarank)"].astype(int)
    df_prediction_test_ranking["予測順位(RankNet)"] = df_prediction_test_ranking["予測順位(RankNet)"].astype(int)
    df_prediction_test_ranking["予測順位(Pairwise)"] = df_prediction_test_ranking["予測順位(Pairwise)"].astype(int)

    # インデックスをリセット
    df_prediction_test_ranking = df_prediction_test_ranking.reset_index(drop=True)

    # 結果を確認
    # print(df_prediction_test_ranking.head())
    # pandasをcsvに保存
    # inference_data_path + /inference_resultにディレクトリを作成し、csvを保存する
    inference_data_path_result = inference_data_path + "inference_result/"
    if not os.path.isdir(inference_data_path_result):
        os.mkdir(inference_data_path_result)
    file_name = file.name
    saving_path = inference_data_path_result + "prediction_second" +  str(file_name) + "_.csv"
    # print("saving_path:{}".format(saving_path))
    df_prediction_test_ranking.to_csv(saving_path, index=False)

    # 推論データに対して予測を行う。1位と2位のスコアの差を計算し、閾値を適用して各モデルごとに1位を予測する
    # df_prediction_test_ranking = prediction_print(df_prediction_test_ranking,optimal_threshold_lambdarank,optimal_threshold_ranknet,optimal_threshold_pairwise,2)
    def print_second_place_info(data, score_column, rank_column):
        # 2位のスコアを取得
        second_place = data[data[rank_column] == 2]
        
        if not second_place.empty:
            for index, row in second_place.iterrows():
                umaban = row['horse_id']
                score = row[score_column]
                rank = row[rank_column]
                result = row['result'] if 'result' in row else 'N/A'
                print(f"Umaban: {umaban}, Score: {score}, Rank: {rank}, Result: {result}")
        else:
            print("No second place found.")
    # それぞれの予測結果、スコアを出力
    print("lambdarank")
    # 2位の情報を出力
    print_second_place_info(df_prediction_test_ranking, "予測スコア(lambdarank)", "予測順位(lambdarank)")
    print("RankNet")
    print_second_place_info(df_prediction_test_ranking, "予測スコア(RankNet)", "予測順位(RankNet)")
    print("Pairwise")
    print_second_place_info(df_prediction_test_ranking, "予測スコア(Pairwise)", "予測順位(Pairwise)")

    find_candidates_for_second_inference(df_prediction_test_ranking, 2)
    # 予測結果(2)の中で2位と予測された馬の情報を出力
    print("Candidates for 2nd place")
    print_second_place_info(df_prediction_test_ranking, "予測スコア(Pairwise)", "予測結果(2)")

    
    count +=1
